In [1]:
y = dbutils.widgets.get("outputFromLookup")
# y = getArgument("outputFromLookup") Another method to get output

In [2]:
import ast
import pandas as pd
#Converting str to dictionary
d = ast.literal_eval(y) 
valueList = []
for i in range(0, len(d['value']), 1):
  valueList.append(d['value'][i])
df_pd = pd.DataFrame(valueList)  

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [4]:
x = df_pd.iloc[:, 2:4]
y = df_pd.iloc[:, 4:5]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

regression = LinearRegression()
regression.fit(x_train, y_train)

y_pred = regression.predict(x_test)
x_test = x_test.reset_index(drop=True)
accuracy = r2_score(y_test, y_pred)
print("Accuracy", accuracy)

In [5]:
y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
result_df = pd.concat([x_test, y_pred_df], axis=1)
print(result_df)

In [6]:
# upload file into blob storage
from azure.storage.blob import *
import pickle

storage_account_name = 'str1wk6'
storage_account_access_key = 'OnTwDpJa14kT8Xi9rY36r9kKeUqNXJdCm1D4RLl9LgUUGBFdmKjQoWhpn2dljkfDAhuDXEprXyB7Ld0eFW+upQ=='
containerName = 'cnt1'
localFileName = '/opt/LinearRegression.pkl'
pickle.dump(regression, open(localFileName, 'wb'))

blob_storage = BlockBlobService(account_name=storage_account_name, account_key=storage_account_access_key)
blob_storage.create_blob_from_path(containerName, 'LinearRegression.pkl', localFileName)